In [ ]:
# import libraries
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas import ExcelWriter
import time
from datetime import datetime
import datetime as dt
import warnings
warnings.simplefilter("ignore")

In [ ]:
# output file name and location
current_date = time.strftime("%Y%m%d")
output_file = 'natural_persons_main_' + current_date + '.xlsx'
output_file_text = 'natural_persons_main_' + current_date + '.txt'

In [ ]:
# get data source
file_path = input("input file path of report 2: ") #

In [ ]:
# read data
df = pd.read_excel(file_path)

In [ ]:
# create text file for links used
with open(output_file_text, 'w') as f:
    f.write('Report path: ' + file_path)
f.close()

***

In [ ]:
# data preprocessing
# select only natural persons
df_natural = df[df['Party Type']=='Natural Person']

In [ ]:
#Preprocessing of party name
df_natural.loc[:,'Party Name_pre']=df_natural['Party Name'].copy()
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.strip()
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace(".","")
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace(" ","")
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace('\t','')
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace(",","")
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.lower()

In [ ]:
#Prepocessing 'Identification Document Number' to remove space & casing
df_natural.loc[:,'Identification Document Number_pre']=df_natural['Identification Document Number'].copy()
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.replace(".","")
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.replace(" ","")
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.replace(",","")
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.lower()

In [ ]:
#dates
df_natural['Date of Birth']  = pd.to_datetime(df_natural['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date

In [ ]:
# create beneficiary only list
bene_only = df_natural.groupby('Party ID')['Relationship (Party Role)'] \
                      .agg(set) \
                      .reset_index(name = 'role_set')

bene_only = bene_only[bene_only['role_set'] == {'Beneficiary'}]
bene_only_list = bene_only['Party ID'].to_list()

df_natural['bene_only'] = df_natural['Party ID'].isin(bene_only_list)

In [ ]:
df_natural['bene_only'].value_counts()

In [ ]:
# add universal columns
df_natural['Batch'] = input("Year, Quarter: (eg. format: 2023 Q1)")
df_natural[['Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception']] = ""

***

In [ ]:
# np1: same name & dob but passport/id country != nationality
# location: sheet 1
tab1 = df_natural.copy()
tab1 = tab1[tab1['bene_only']==False]

#create list of Party IDs with HK permanent ID
HK_permID = tab1[tab1['Identification Document Type'] == 'HK Permanent ID']
HK_permID = HK_permID.drop_duplicates(subset = ['Party ID'])
HK_permID_list = HK_permID['Party ID'].tolist()

#create list of British Overseas Territories Passports (obtained from google)
Brit_set = {'Anguilla','Bermuda','Virgin Islands','Cayman Islands','Gibraltar','Guernsey','Jersey','Manx','Montserrat',
            'Saint Helena','Turks and Calcos Islands','Isle Of Man'}

# identification Document Type ==  'Passport'
tab1 = tab1[tab1['Identification Document Type'] == 'Passport']

#create group identifier: [key]
tab1['key'] = tab1['Party Name_pre'].astype(str) + \
              tab1['Date of Birth'].astype(str)

#Replace Brit Overseas Territories Passports with UK passports
tab1_Brit = tab1.copy()
'''"""tab1_Brit.loc[tab1['Identification Document Issue Country'].isin(Brit_set), 
              'Identification Document Issue Country'] = "United Kingdom""""'''

# create group identifier: [passport key]
tab1['passport_key'] = tab1_Brit['Party Name_pre'].astype(str) + \
                       tab1_Brit['Date of Birth'].astype(str) + \
                       tab1_Brit['Identification Document Issue Country'].astype(str)

# create group identifier: [nationality key]
tab1['nationality_key'] = tab1['Party Name_pre'].astype(str) + \
                          tab1['Date of Birth'].astype(str) + \
                          tab1['Nationality'].astype(str)



#create list of nationalities
tab1_nationality_list = tab1.nationality_key.tolist()
#check if passports can be found in list of nationalities, flag if cannot be found
tab1['flag'] = ~(tab1['passport_key'].isin(tab1_nationality_list))

#create list of passports
tab1_pp_list = tab1.passport_key.tolist()
#check if nationalities can be found in list of passports, flag if cannot be found
tab1['flag2'] = ~(tab1['nationality_key'].isin(tab1_pp_list))

#filter for records where passports cannot be found in nationalities and vice versa
tab1_filter = tab1[(tab1['flag'] == True) | 
                   (tab1['flag2'] == True)]
          
#create list of parties to flag based on "key" [party name + DOB]
tab1_filter_list = tab1_filter.key.tolist()

#retrieve all records that is in list of parties to flag
tab1_columns = tab1[(tab1['key'].isin(tab1_filter_list))]

#indicate if party ID has HK Perm ID, if nationality- HK
tab1_columns['HK Permanent ID?'] = ""
tab1_columns.loc[((tab1_columns['Party ID'].isin(HK_permID_list)) & 
                  (tab1_columns['Nationality'] == "Hong Kong")),
                 'HK Permanent ID?'] = 'True'

# state inconsistency type in new column 'Inconsistency'
tab1_columns['Inconsistency'] = 'Same Name & DOB but Passport ID Doc Ctry <> Nationality'

#drop duplicates
tab1_columns = tab1_columns.drop_duplicates(subset = ['Party ID',
                                                      'Identification Document Type',
                                                      'Identification Document Issue Country',
                                                      'Nationality'])

In [ ]:
# np2: same name & dob but nationality != id proof (SG, Swiss, Italian, Germany, French, Liechtenstein, Thailand)
# location: sheet 1
tab2 = df_natural.copy()
tab2 = tab2[tab2['bene_only']==False]

# create group identifier: [key]
tab2['key'] = tab2['Party Name_pre'].astype(str) + \
              tab2['Date of Birth'].astype(str)

# 2.1 checking for sg pink NRIC
tab2_1 = tab2[tab2['Identification Document Type'] == 'SG Pink NRIC']

# create a [Nationality] set for each group
tab2_nationality = tab2_1.groupby('key')['Nationality'] \
                        .agg(set) \
                        .reset_index(name = 'nationality_set')

nationality_set = tab2_nationality.nationality_set.tolist()

# check if Singapore exist in [Nationality]
def check_singapore(nationality_set): #check if Singapore exists in the set of Nationality
    singapore_set = {'Singapore'}
    result = nationality_set.difference(singapore_set)
    if 'Singapore' in nationality_set: return 'correct'
    else: return result
    
tab2_nationality['check_singapore'] = tab2_nationality['nationality_set'].apply(lambda x: check_singapore(x))
tab2_1_error = tab2_nationality[tab2_nationality['check_singapore'] != 'correct']

# 2.2 checking for national id of countries
# select rows  with national id of countries
nationality_list = ['Switzerland', 'Italy', 'Germany', 'France', 'Liechtenstein', 'Thailand']
tab2_2 = tab2[tab2['Identification Document Type'] == 'National ID']
tab2_2_filter = tab2_2[(tab2_2['Identification Document Issue Country'].isin(nationality_list))]

# create a [Nationality] set for each group
tab2_2_nationality = tab2_2_filter.groupby('key')['Nationality'] \
                                  .agg(set) \
                                  .reset_index(name='nationality_set')

tab2_nationality_set = tab2_2_nationality.nationality_set.tolist()

# create a [Identification Document Issue Country] set for each group
tab2_2_idctry = tab2_2_filter.groupby('key')['Identification Document Issue Country'] \
                             .agg(set) \
                             .reset_index(name='idctry_set')

tab2_2_idctry['nationality_set'] = tab2_nationality_set

# compare two sets: set(idcrty) - set(nationality)
def compare_country_sets_reverse(idctry_set, nationality_set): #compare difference btw {ID doc ctry} and {nationality}
    result = idctry_set.difference(nationality_set)
    if result: return result #if result not empty, return result
    else: return 'correct' #if result IS empty, return 'correct'

# extract groups whose sets is not identical     
tab2_2_idctry['set_diff'] = tab2_2_idctry.apply(lambda x:
                                                compare_country_sets_reverse(x.idctry_set, x.nationality_set), 
                                                axis = 1)
tab2_2_error = tab2_2_idctry[tab2_2_idctry['set_diff'] != 'correct']

# v-lookup in originial df 
tab2_1_error_list = tab2_1_error['key'].tolist()
tab2_2_error_list = tab2_2_error['key'].tolist()
tab2_error_list = tab2_1_error_list + tab2_2_error_list

tab2['exist'] = tab2['key'].isin(tab2_error_list)
tab2_columns = tab2[tab2['exist'] == True]

# state inconsistency type in new column 'Inconsistency'
tab2_columns['Inconsistency'] = \
'Same Name & DOB but Nationality <> ID proof(SG, Swiss, Italian, Germany, French, Liechtenstein)'

#drop duplicates
tab2_columns = tab2_columns.drop_duplicates(subset=['Party ID','Identification Document Type',
                                                    'Identification Document Issue Country','Nationality'])

In [ ]:
# np4: same name & dob but diff place of birth
# location: sheet 1
tab4 = df_natural.copy()
tab4 = tab4[tab4['bene_only']==False]

# create concat column of same party name & same DOB
tab4['key'] = tab4['Party Name_pre'].astype(str) + \
              tab4['Date of Birth'].astype(str)

# add a column to count no. of Place of Birth based on same Party Name & DOB
tab4['unique_count'] = tab4.groupby(['key'])['Place of Birth'] \
                           .transform('nunique')

# keep only rows where theres > 1 Place of Birth 
tab4 = tab4[tab4['unique_count'] > 1]

# remove duplicates
tab4_columns = tab4.drop_duplicates(['Party ID','key','Place of Birth'])

# state inconsistency type in new column 'Inconsistency'
tab4_columns['Inconsistency'] = 'Same Name, DOB, Diff Place of Birth'

In [ ]:
# np5: same name & dob & address type but diff address country
# location: sheet 1
tab5 = df_natural.copy()
tab5 = tab5[tab5['bene_only']==False]
tab5 = tab5[tab5['Address Type'] == 'Existing Residential Address']

# create concat column of same party name, DOB
tab5['key'] = tab5['Party Name_pre'].astype(str) + tab5['Date of Birth'].astype(str)

# add a column to count no. of Address - Country based on same Party Name & DOB
tab5['unique_count'] = tab5.groupby(['key'])['Address - Country'] \
                           .transform('nunique')

# keep only rows where theres > 1 Address - Country 
tab5 = tab5[tab5['unique_count'] > 1]

# remove duplicates
tab5_columns = tab5.drop_duplicates(subset=['Party ID','key','Address - Country'])

# state inconsistency type in new column 'Inconsistency'
tab5_columns['Inconsistency'] = 'Same Name, DOB, Diff Address Country'

In [ ]:
# np6: same name & dob & id# but diff id doc type
# location: sheet 1
tab6 = df_natural.copy()
tab6 = tab6[tab6['bene_only']==False]

# create concat column of same party name, DOB, Identification Document Number
tab6['key'] = tab6['Party Name_pre'].astype(str) + \
              tab6['Date of Birth'].astype(str) + \
              tab6['Identification Document Number_pre'].astype(str)

# add a column to count no. of Identification Document Type based on same party name, DOB & Identification Document Number
tab6['unique_count'] = tab6.groupby(['key'])['Identification Document Type'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Type 
tab6 = tab6[tab6['unique_count'] > 1]

# remove duplicates
tab6_columns = tab6.drop_duplicates(subset=['Party ID','key','Identification Document Type'])

# state inconsistency type in new column 'Inconsistency'
tab6_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Doc Type'

In [ ]:
# np7: same name & dob & id# but diff id expiry
# location: sheet 1
tab7 = df_natural.copy()
tab7 = tab7[tab7['bene_only']==False]

#Treat ID Document Expiry Date as a string
tab7['Identification Document Expiry Date_1'] = tab7['Identification Document Expiry Date'].astype(str)

# create concat column of same party name, DOB, Identification Document Number
tab7['key'] = tab7['Party Name_pre'].astype(str) + \
              tab7['Date of Birth'].astype(str) + \
              tab7['Identification Document Number_pre'].astype(str)

# add a column to count no. of Identification Document Expiry Date based on same party name, DOB & Identification Document Number
tab7['unique_count'] = tab7.groupby(['key'])['Identification Document Expiry Date_1'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Expiry Date 
tab7 = tab7[tab7['unique_count'] > 1]

#groupby party name/DOB/ID and party RM (RM UID.1)
tab7['unique_count_party'] = tab7.groupby(['key'])['RM UID.1'] \
                                 .transform('nunique')


# remove duplicates
tab7_columns = tab7.drop_duplicates(subset = ['Party ID','key','Identification Document Expiry Date_1'])

# state inconsistency type in new column 'Inconsistency'
tab7_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Expiry'

In [ ]:
# np8: same name & dob & id# but diff id country
# location: sheet 1
tab8 = df_natural.copy()
tab8 = tab8[tab8['bene_only']==False]

# create concat column of same party name, DOB, Identification Document Number
tab8['key'] = tab6['Party Name_pre'].astype(str) + \
              tab8['Date of Birth'].astype(str) + \
              tab8['Identification Document Number_pre'].astype(str)

# add a column to count no. of Identification Document Issue Country based on same party name, DOB & Identification Document Number
tab8['unique_count'] = tab8.groupby(['key'])['Identification Document Issue Country'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Issue Country 
tab8 = tab8[tab8['unique_count'] > 1]

# remove duplicates
tab8_columns = tab8.drop_duplicates(subset=['key','Identification Document Issue Country'])

# state inconsistency type in new column 'Inconsistency'
tab8_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Doc Country'

In [ ]:
# np9: same name & dob & id# but diff id issue date
# location: sheet 1
tab9 = df_natural.copy()
tab9 = tab9[tab9['bene_only']==False]

# create concat column of same party name, DOB, Identification Document Number
tab9['key'] = tab9['Party Name_pre'].astype(str) + \
              tab9['Date of Birth'].astype(str) + \
              tab9['Identification Document Number_pre'].astype(str)

#Treat ID date of issue as a string
tab9['Identification Date of Issue_1'] = tab9['Identification Date of Issue'].astype(str)


# add a column to count no. of Identification Date of Issue based on same party name, DOB & Identification Document Number
tab9['unique_count'] = tab9.groupby(['key'])['Identification Date of Issue_1'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Date of Issue 
tab9 = tab9[tab9['unique_count'] > 1]

#groupby party name/DOB/ID and party RM (RM UID.1)
tab9['unique_count_party'] = tab9.groupby(['key'])['RM UID.1'] \
                                 .transform('nunique')

# remove duplicates
tab9_columns = tab9.drop_duplicates(subset = ['key','Identification Date of Issue_1'])

# state inconsistency type in new column 'Inconsistency'
tab9_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Issue date'

In [ ]:
# np10: same name & dob & id type & id coountry but diff id#
# location: sheet 1
tab10 = df_natural.copy()
tab10 = tab10[tab10['bene_only']==False]

# create concat column of same party name, DOB, Identification Document Type, Identification Document Issue Country 
tab10['key'] = tab10['Party Name_pre'].astype(str) + \
               tab10['Date of Birth'].astype(str) + \
               tab10['Identification Document Type'].astype(str) + \
               tab10['Identification Document Issue Country'].astype(str)

# add a column to count no. of Identification Document Number based on same party name, DOB, Identification Document Type,Identification Document Issue Country
tab10['unique_count'] = tab10.groupby(['key'])['Identification Document Number_pre'] \
                             .transform('nunique')

# keep only rows where theres > 1 Identification Document Number 
tab10 = tab10[tab10['unique_count'] > 1]

# remove duplicates
tab10_columns = tab10.drop_duplicates(subset = ['Party ID','key','Identification Document Number_pre'])

# state inconsistency type in new column 'Inconsistency'
tab10_columns['Inconsistency'] = 'Same Name, DOB, ID Doc Type, ID Doc Country, Diff ID'

In [ ]:
# np11: same name & dob & party rm but diff party id
# location: sheet 1
tab11 = df_natural.copy()

# create concat column of same party name, DOB, Party RM
tab11['key'] = tab11['Party Name_pre'].astype(str) + \
               tab11['Date of Birth'].astype(str) + \
               tab11['RM Name.1'].astype(str)

# add a column to count no. of Party ID based on same party name, DOB & Party RM
tab11['unique_count'] = tab11.groupby(['key'])['Party ID'] \
                             .transform('nunique')

# keep only rows where theres > 1 Party ID 
tab11 = tab11[tab11['unique_count'] > 1]

# remove duplicates
tab11_columns = tab11.drop_duplicates(subset = ['key','Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab11_columns['Inconsistency'] = 'Same Name, DOB, Party RM, Diff Party ID'

In [ ]:
# np12: same name & same id type & id# but diff dob 
# location: sheet 1
tab12 = df_natural.copy()
tab12 = tab12[tab12['bene_only']==False]

#Treat DOB as a string
tab12['Date of Birth_1'] = tab12['Date of Birth'].astype(str)

# create concat column of same party name, DOB, Party RM
tab12['key'] = tab12['Party Name_pre'].astype(str) + \
               tab12['Identification Document Type'].astype(str) + \
               tab12['Identification Document Number'].astype(str)

# add a column to count no. of Party ID based on same party name, DOB & Party RM
tab12['unique_count'] = tab12.groupby(['key'])['Date of Birth_1'] \
                             .transform('nunique')

# keep only rows where theres > 1 Party ID 
tab12 = tab12[tab12['unique_count'] > 1]

# remove duplicates
tab12_columns = tab12.drop_duplicates(subset = ['key','Date of Birth'])

# state inconsistency type in new column 'Inconsistency'
tab12_columns['Inconsistency'] = 'Same Name, Same ID Doc Type, ID #, Diff DoB'

In [ ]:
# np13: Same ID Doc Type, ID #, ID Doc Country, Diff Name
# location: sheet 2
tab13 = df_natural.copy()
tab13 = tab13[tab13['bene_only']==False]

# Check if Identification Document Number contains invalid value
tab13['invalid_identification_document_number'] = tab13['Identification Document Number'].isin(['[]','()', np.nan,'NA', 
                                                                                                'NA in iCare', 
                                                                                                'N.A In Icare', 
                                                                                                'n/a','na','N/A','N.A.',
                                                                                                'N.A','indeterminata',
                                                                                                'Indeterminata',' ','','.',
                                                                                                '-','0','0000000','000000',
                                                                                                'not available', 
                                                                                                'not applicable', 
                                                                                                'NIL', 'xx', 'not','XX', 
                                                                                                'Not available',
                                                                                                '[Not Available]',
                                                                                                '[Un Specified]', 'Na'])


#Filter for records where ID Doc Type / ID# / ID Doc Country is not blank or invalid
tab13 = tab13[(tab13['Identification Document Type'] != "") & 
              (tab13['invalid_identification_document_number'] == False) &
              (tab13['Identification Document Issue Country'] != "")]

# create concat column
tab13['key'] = tab13['Identification Document Type'].astype(str) + \
               tab13['Identification Document Issue Country'].astype(str) + \
               tab13['Identification Document Number_pre'].astype(str)

# add a column to count no. of party name
tab13['unique_count'] = tab13.groupby(['key'])['Party Name_pre'] \
                             .transform('nunique')

# keep only rows where theres > 1 name
tab13 = tab13[tab13['unique_count'] > 1]

# drop duplicates
#tab13 = tab12.sort_values(by=['Party Name_pre'])
tab13_columns = tab13.drop_duplicates(subset = ['Party ID','key','Party Name_pre'])

# state inconsistency type in new column 'Inconsistency'
tab13_columns['Inconsistency'] = 'Same ID Doc Type, ID #, ID Doc Country, Diff Name'

In [ ]:
# np14: DON expiry – issue date > 2 years or missing
# location: sheet 3
tab14 = df_natural.copy()
tab14 = tab14[tab14['bene_only']==False]

# filter for Identification Document Type == Declaration of Nationality
tab14 = tab14[tab14['Identification Document Type'] == 'Declaration of Nationality']

# calculate time delta between Identification Document Expiry Date & Identification Document Expiry Date
tab14['days_diff'] = tab14['Identification Document Expiry Date'] - tab14['Identification Date of Issue']

# filter for days_diff > 731
tab14_columns = tab14[(tab14['days_diff'] > dt.timedelta(731)) | 
                      (tab14['days_diff'].isna())]

# state inconsistency type in new column 'Inconsistency'
tab14_columns['Inconsistency'] = 'DON expiry – issue date > 2 years or missing'

In [ ]:
# np15: Invalid Place of Birth
# location: sheet 3
tab15 = df_natural.copy()
tab15 = tab15[tab15['bene_only']==False]
tab15 = tab15[(tab15['Identification Document Type']=='Passport') & 
              (~tab15['Identification Document Issue Country'].isin(['Japan','Korea','Switzerland','Saudi Arabia', 'Canada',
                                                                    'Italy', 'Germany', 'France', 'Princip.Liechtenstein',
                                                                    'Republic Of Korea',"insert country here"]))]

# extract rows which Place of Birth contains invalid value
tab15['invalid_place_of_birth'] = tab15['Place of Birth'].isin(['[]','()', np.nan,'NA', 'NA in iCare', 'N.A In Icare', 
                                                                'n/a','na','N/A','N.A.','N.A','indeterminata',
                                                                'Indeterminata',' ','','.','-','0','0000000',
                                                                '000000','not available', 'not applicable', 
                                                                'NIL', 'xx', 'not','XX', 'Not available','[Not Available]',
                                                                '[Un Specified]', 'Europe', 'International'])
tab15_filter = tab15[tab15['invalid_place_of_birth'] == True]

tab15_columns = tab15_filter.drop_duplicates(subset = ['Party ID'])

tab15_columns['Inconsistency'] = 'Invalid Place of Birth'

In [ ]:
# np16: Invalid ID Doc Number
# location: sheet 3
tab16 = df_natural.copy()
tab16 = tab16[tab16['bene_only']==False]

# extract rows which Identification Document Number contains invalid value
tab16['invalid_identification_document_number'] = tab16['Identification Document Number'].isin(['[]','()', np.nan,'NA', 
                                                                                                'NA in iCare', 
                                                                                                'N.A In Icare', 
                                                                                                'n/a','na','N/A','N.A.',
                                                                                                'N.A','indeterminata',
                                                                                                'Indeterminata',' ','','.',
                                                                                                '-','0','0000000','000000',
                                                                                                'not available', 
                                                                                                'not applicable', 
                                                                                                'NIL', 'xx', 'not','XX', 
                                                                                                'Not available',
                                                                                                '[Not Available]',
                                                                                                '[Un Specified]'])
tab16_filter = tab16[(tab16['invalid_identification_document_number'] == True) &
                     (tab16['Identification Document Type']!='Declaration of Nationality')]

tab16_columns = tab16_filter.drop_duplicates(subset = ['Party ID',
                                                       'Identification Document Type',
                                                       'Identification Document Issue Country',
                                                       'Identification Document Number'])

tab16_columns['Inconsistency'] = 'Invalid ID Doc #'

In [ ]:
# np17: ID Doc Issue Country is blank when ID doc type is not blank
# location: sheet 3
tab17 = df_natural.copy()
tab17 = tab17[tab17['bene_only']==False]

tab17_filter = tab17[~tab17['Identification Document Type'].isna()]

tab17_filter_2 = tab17_filter[tab17_filter['Identification Document Issue Country'].isna()]

tab17_columns = tab17_filter_2.drop_duplicates(subset = ['Party ID',
                                                         'Identification Document Type',
                                                         'Identification Document Issue Country'])

# state inconsistency type in new column 'Inconsistency'
tab17_columns['Inconsistency'] = 'ID Doc Issue Country is blank when ID doc type is not blank'

In [ ]:
# np18: Party ID tagged to 2 portfolio RMs (Party Role = AH & BO only)
# location: sheet 3
tab18 = df_natural.copy()
tab18 = tab18[tab18['bene_only']==False]

tab18_filter = tab18[(tab18['Relationship (Party Role)'] == 'Account Holder') |
                     (tab18['Relationship (Party Role)'] == 'Beneficial Owner')]

# add a column to count no. of party name
tab18_filter['unique_count'] = tab18_filter.groupby(['Party ID'])['RM UID'] \
                                           .transform('nunique')

# keep only rows where theres > 1 party ID 
tab18_filter = tab18_filter[tab18_filter['unique_count'] > 1]

#drop duplicates
tab18_columns = tab18_filter.drop_duplicates(subset = ['Party ID','RM UID'])

# state inconsistency type in new column 'Inconsistency'
tab18_columns['Inconsistency'] = 'Party ID tagged to 2 portfolio RMs (Party Role = AH & BO only)'

In [ ]:
# np22: ID only and not from acceptable countries
# location: sheet 1
tab22 = df_natural.copy()
tab22 = tab22[tab22['bene_only']==False]

# create group identifier: [key]
tab22['key'] = tab22['Party Name_pre'].astype(str) + tab22['Date of Birth'].astype(str)

# count of ID doc type based on same name and DOB
tab22['unique_count'] = tab22.groupby(['key'])['Identification Document Type'] \
                             .transform('nunique')

# keep only rows where theres only 1 ID doc type and the ID doc type is ID/NRIC (pink/blue with expiry date)
tab22_1 = tab22[(tab22['unique_count'] == 1) & (tab22['Identification Document Type'] == 'SG Pink NRIC')]
tab22_2 = tab22[(tab22['unique_count'] == 1) & (tab22['Identification Document Type'] == 'National ID')]
tab22_3 = tab22[(tab22['Identification Document Type'] == 'SG Blue NRIC') & (tab22['Identification Document Expiry Date'].notnull())]

# Check if Nationality = Singaporean
tab22_1_error = tab22_1[tab22_1['Nationality'] != 'Singapore']

# Check if Nationality = acceptable countries
# National ID AND Nationality = acceptable countries --> this should detect parties with SG nationality and National ID
country_list = ['Switzerland', 'Italy', 'Germany', 'France', 'Princip.Liechtenstein', 'Thailand']
tab22_2_error = tab22_2[~tab22_2['Nationality'].isin(country_list)]

# flag if
# 1. Blue NRIC with Expiry Date and Nationality==Singapore
# 2. Blue NRIC with Expiry Date and Nationality > 1
tab22_3['unique_count'] = tab22.groupby(['key'])['Nationality'].transform('nunique')
tab22_3_error = tab22_3[((tab22_3['unique_count']==1) & (tab22_3['Nationality']=='Singapore')) | (tab22_3['unique_count']>1)]


# v-lookup in originial df 
tab22_1_error_list = tab22_1_error['key'].tolist()
tab22_2_error_list = tab22_2_error['key'].tolist()
tab22_3_error_list = tab22_3_error['key'].tolist()
tab22_error_list = tab22_1_error_list + tab22_2_error_list + tab22_3_error_list

tab22['exist'] = tab22['key'].isin(tab22_error_list)
tab22_columns = tab22[tab22['exist'] == True]

# state inconsistency type in new column 'Inconsistency'
tab22_columns['Inconsistency'] = 'ID only and not from acceptable countries'

#drop duplicates
tab22_columns = tab22_columns.drop_duplicates(subset = ['Party ID', 'Nationality'])

In [ ]:
# np23: Blue NRIC not SG Resident
# location: sheet 1
tab23 = df_natural.copy()
tab23 = tab23[tab23['bene_only']==False]

# create group identifier: [key]
tab23['key'] = tab23['Party Name_pre'].astype(str) + tab23['Date of Birth'].astype(str)

# count of ID doc type based on same name and DOB
tab23['unique_count'] = tab23.groupby(['key'])['Identification Document Type'] \
                             .transform('nunique')

# keep only rows where theres only 1 ID doc type and the ID doc type is Blue NRIC
tab23 = tab23[(tab23['unique_count'] == 1) & 
              (tab23['Identification Document Type'] == 'SG Blue NRIC')]

# Residential Pass Holder of = Singapore
# Address type = Existing Residential Address 
# Address - Country = Singapore
tab23_columns = tab23[~((tab23["Resident pass holder of"] == 'Singapore') &
                        (tab23["Address Type"] == 'Existing Residential Address') &
                        (tab23["Address - Country"] == 'Singapore')
                       )
                     ]

# state inconsistency type in new column 'Inconsistency'
tab23_columns['Inconsistency'] = 'Blue NRIC not SG Resident'

#drop duplicates
tab23_columns = tab23_columns.drop_duplicates(subset = ['Party ID', 'Nationality'])

In [ ]:
# np24: Invalid override expiry reason
# location: sheet 5
tab24 = df_natural.copy()
tab24 = tab24[tab24['bene_only']==False]

#create group identifier: [key]
tab24['key']=tab24['Party Name_pre'].astype(str) + tab24['Date of Birth'].astype(str)

doc_types = ['SG Pink NRIC', 'National ID', 'Passport']
tab24 = tab24[tab24["Identification Document Type"].isin(doc_types)]

tab24 = tab24[(tab24['Override Expiry'] == 'Yes') & (tab24['Override Expiry Reason'] != 'Deceased')]

tab24_columns = tab24.drop_duplicates(subset = ['Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab24_columns['Inconsistency'] = 'Invalid override expiry reason'

In [ ]:
# np25: override reason and deceased flag mismatch
# location: sheet 5
tab25 = df_natural.copy()
tab25 = tab25[tab25['bene_only']==False]

#create group identifier: [key]
tab25['key']=tab25['Party Name_pre'].astype(str) + tab25['Date of Birth'].astype(str)

doc_types = ['SG Pink NRIC', 'National ID', 'Passport']
tab25 = tab25[tab25["Identification Document Type"].isin(doc_types)]

tab25 = tab25[
              ((tab25['Override Expiry']=='Yes')&(tab25['Override Expiry Reason']=='Deceased')&(tab25['Deceased Flag']!='Yes'))
             |((tab25['Override Expiry']=='Yes')&(tab25['Override Expiry Reason']!='Deceased')&(tab25['Deceased Flag']=='Yes'))
             |((tab25['Override Expiry']!='Yes')&(tab25['Override Expiry Reason']=='Deceased')&(tab25['Deceased Flag']=='Yes'))
             |((tab25['Override Expiry']!='Yes')&(tab25['Override Expiry Reason']!='Deceased')&(tab25['Deceased Flag']=='Yes'))
             |((tab25['Override Expiry']!='Yes')&(tab25['Override Expiry Reason']=='Deceased')&(tab25['Deceased Flag']!='Yes'))
             ]


tab25_columns = tab25.drop_duplicates(subset = ['Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab25_columns['Inconsistency'] = 'Override and Deceased Flag Mismatch'

In [ ]:
# np26: invalid ID doc type
# location: sheet 1
# flag only:
# HK ID, HK Permanent ID/dependent/employment pass/s pass/social pass/student pass/green card
# SG Blue NRIC without expiry date
tab26 = df_natural.copy()
tab26 = tab26[tab26['bene_only']==False] # bene only false
tab26 = tab26[tab26['Deceased Flag']!='Yes'] # not deceased 

# create group identifier: [key]
tab26['key'] = tab26['Party Name_pre'].astype(str) + tab26['Date of Birth'].astype(str)

# count of ID doc type based on same name and DOB
tab26['unique_count'] = tab26.groupby(['key'])['Identification Document Type'] \
                             .transform('nunique')

# parties with only 1 id doc?
# tab26 = tab26[tab26['unique_count']==1]
# should be parties w/o national ID/passport
doc_types = ['SG Pink NRIC', 'National ID', 'Passport']
exclude_parties = tab26[tab26["Identification Document Type"].isin(doc_types)]['Party ID'].tolist()
tab26 = tab26[~tab26["Party ID"].isin(exclude_parties)]


# part 1 filtering for obvious invalid ID/others/blank
invalid_id_list = ['HK ID', 'HK Permanent ID', 'Dependent Pass', 'Employment Pass', 'S Pass', 'Social Pass', 'Student Pass',
                   'Green Card']
tab26_1 = tab26[(tab26['Identification Document Type'].isin(invalid_id_list)) |
                (tab26['Identification Document Type'].isna())]


# part 2 blue NRIC and no expiry date
tab26_2 = tab26[(tab26['Identification Document Type'] == 'SG Blue NRIC') &
                (tab26['Identification Document Expiry Date'].isna())]

# part 3 ID not from acceptable country (take from tab22)
#tab26_3 = tab26[tab26.index.isin(tab22_columns.index)]

# part 4 ID doc type = 'Other'
tab26_4 = tab26[tab26['Identification Document Type'] == 'Other']

# part 5 Singapore Nationality with "national ID"
# should be covered in 22
#tab26_5 = tab26[(tab26['Nationality'] == 'Singapore') & (tab26['Identification Document Type'] == 'National ID')]

# join then drop duplicates
tab26_columns = pd.concat([tab26_1, tab26_2, tab26_4])
tab26_columns = tab26_columns.drop_duplicates(subset = ['Party ID', 'Nationality'])

# state inconsistency type in new column 'Inconsistency'
tab26_columns['Inconsistency'] = 'Invalid ID '

In [ ]:
# tab 27 - letter of undertaking + valid ID doc - flag for cleanup to remove letter of undertaking from CRM
# sheet 1
tab27 = df_natural.copy()
#tab27 = tab27[tab27['bene_only']==False]
#tab27 = tab27[tab27['Identification Document Type']=='Other']

# first clean up column so we have an easier time searching
tab27['Identification Document - Please Specify'] = tab27['Identification Document - Please Specify'].str.lower()


tab27 = tab27.fillna('0') # can only use .str.contains() if there are no null values

# regex search to find those that contain any variation of the phrase 'letter of undertaking'
lou = ['letter of undertaking', 'lou', 'letter', 'undertaking']
pat = '|'.join(lou)
tab27 = tab27[tab27['Identification Document - Please Specify'].str.contains(pat)]

# end
tab27_columns = tab27.drop_duplicates(subset = ['Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab27_columns['Inconsistency'] = 'ID Docs Contains Letter of Undertaking'

***

In [ ]:
# sheet 1
df_1 = [tab1_columns,tab2_columns,tab4_columns,tab5_columns,tab6_columns,tab7_columns,tab8_columns,tab9_columns,
        tab10_columns,tab11_columns,tab12_columns,tab22_columns, tab23_columns, tab26_columns]

result_1 = pd.concat(df_1, join='outer', axis=0)
result_1 = result_1.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_1['Beneficiary_role_only']=result_1['Party ID'].isin(bene_only_list)

# select columns
result_1 = result_1[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','RM UID.1','RM Name.1','Date of Birth','Place of Birth','Resident pass holder of',
                    'Address Type','Address - Country','Nationality','Identification Document Type','Identification Document Issue Country',
                    'Identification Document Number','Identification Date of Issue','Identification Document Expiry Date',
                    'Identification Document - Please Specify','Beneficiary_role_only','HK Permanent ID?']]

#rename columns
result_1.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Date of Birth','Place of Birth','Resident pass holder of',
                    'Address Type','Address - Country','Nationality','Identification Document Type','Identification Document Issue Country',
                    'Identification Document Number','Identification Date of Issue','Identification Document Expiry Date',
                    'Identification Document - Please Specify','Beneficiary_role_only','HK Permanent ID?']

In [ ]:
# sheet 2
result_2 = tab13_columns.sort_values(by=['key','Party Name_pre','Party ID'])
result_2['Beneficiary_role_only']=result_2['Party ID'].isin(bene_only_list)

# select columns
result_2 = result_2[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','RM UID.1','RM Name.1','Date of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Beneficiary_role_only']]
# rename columns
result_2.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Date of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Beneficiary_role_only']

In [ ]:
# sheet 3
df_3 = [tab14_columns,tab15_columns,tab16_columns,tab17_columns,tab18_columns]

result_3 = pd.concat(df_3, join='outer', axis=0)
result_3 = result_3.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_3['Beneficiary_role_only']=result_3['Party ID'].isin(bene_only_list)

# select columns
result_3 = result_3 [['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','RM UID.1','RM Name.1','RM UID','RM Name','Date of Birth','Place of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Identification Date of Issue','Identification Document Expiry Date','Identification Document - Please Specify',
                    'Beneficiary_role_only']]

# rename columns
result_3.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Portfolio RM UID','Portfolio RM Name','Date of Birth','Place of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Identification Date of Issue','Identification Document Expiry Date','Identification Document - Please Specify',
                    'Beneficiary_role_only']

In [ ]:
# sheet 5
df_5 = [tab24_columns,tab25_columns]

result_5 = pd.concat(df_5, join='outer', axis=0)
result_5 = result_5.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_5['Beneficiary_role_only']=result_5['Party ID'].isin(bene_only_list)

# select columns
result_5 = result_5[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID', 'Party Name','RM UID.1','RM Name.1','Date of Birth',
                    'Identification Document Type','Identification Document Number','Identification Document Issue Country',
                    'Identification Date of Issue','Identification Document Expiry Date',
                    'Override Expiry','Override Expiry Reason','Override Expiry Justification',
                    'Deceased Flag','Beneficiary_role_only']]

# rename columns
result_5.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID', 'Party Name','Party RM UID','Party RM Name','Date of Birth',
                    'Identification Document Type','Identification Document Number','Identification Document Issue Country',
                    'Identification Date of Issue','Identification Document Expiry Date',
                    'Override Expiry','Override Expiry Reason','Override Expiry Justification',
                    'Deceased Flag','Beneficiary_role_only']

In [ ]:
# sheet 6
df_6 = [tab27_columns]

result_6 = pd.concat(df_6, join='outer', axis=0)
result_6 = result_6.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_6['Beneficiary_role_only']=result_6['Party ID'].isin(bene_only_list)

# select columns
result_6 = result_6[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','RM UID.1','RM Name.1','Date of Birth','Place of Birth','Resident pass holder of',
                    'Address Type','Address - Country','Nationality','Identification Document Type','Identification Document Issue Country',
                    'Identification Document Number','Identification Date of Issue','Identification Document Expiry Date',
                    'Identification Document - Please Specify','Beneficiary_role_only']]

#rename columns
result_6.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Date of Birth','Place of Birth','Resident pass holder of',
                    'Address Type','Address - Country','Nationality','Identification Document Type','Identification Document Issue Country',
                    'Identification Document Number','Identification Date of Issue','Identification Document Expiry Date',
                    'Identification Document - Please Specify','Beneficiary_role_only']

In [ ]:
writer = ExcelWriter(output_file, mode='w',date_format = 'yyyy-mm-dd', 
                        datetime_format='yyyy-mm-dd')

result_1.to_excel(writer, 'Same_name_diff_static_info', index=False) #tab1
result_2.to_excel(writer, 'Same_ID_diff_name', index=False) #tab2
result_3.to_excel(writer, 'incomplete_fields_or_2RM', index=False) #tab3
result_5.to_excel(writer, 'Invalid_override_expiry', index=False) #tab5
result_6.to_excel(writer, 'Letter of undertaking cleanup', index=False) #tab6

writer.save()

In [ ]:
print("Completed.")